In [5]:
%%writefile listDegree.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <string>

#define N 5 // Number of nodes in the graph

__global__ void computeDegree(int *adjMatrix, int *degree) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {
        int count = 0;
        for (int i = 0; i < N; ++i) {
            count += adjMatrix[id * N + i];
        }
        degree[id] = count;
    }
}

int main() {
    // Read adjacency matrix from CSV file
    std::ifstream file("adjacency_matrix.csv");
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file!" << std::endl;
        return 1;
    }

    std::vector<std::vector<int>> adjMatrix(N, std::vector<int>(N, 0));
    std::string line;
    int row = 0;
    while (std::getline(file, line) && row < N) {
        std::stringstream ss(line);
        std::string cell;
        int col = 0;
        while (std::getline(ss, cell, ',') && col < N) {
            adjMatrix[row][col] = std::stoi(cell);
            col++;
        }
        row++;
    }

    // Print the read adjacency matrix for debugging
    std::cout << "Adjacency Matrix:" << std::endl;
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            std::cout << adjMatrix[i][j] << " ";
        }
        std::cout << std::endl;
    }

    // Allocate memory for adjacency matrix and degree array
    int *adjMatrix_dev, *degree_dev;
    cudaMallocManaged(&adjMatrix_dev, N * N * sizeof(int));
    cudaMallocManaged(&degree_dev, N * sizeof(int));

    // Copy adjacency matrix to device memory
    for (int i = 0; i < N; ++i) {
        cudaMemcpy(adjMatrix_dev + i * N, &adjMatrix[i][0], N * sizeof(int), cudaMemcpyHostToDevice);
    }

    // Launch kernel to compute degree
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    computeDegree<<<numBlocks, blockSize>>>(adjMatrix_dev, degree_dev);
    cudaDeviceSynchronize();

    // Copy degrees from device to host
    int degree[N];
    cudaMemcpy(degree, degree_dev, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print out degrees
    std::cout << "Node degrees:" << std::endl;
    for (int i = 0; i < N; ++i) {
        std::cout << "Node " << i << ": " << degree[i] << std::endl;
    }

    // Free memory
    cudaFree(adjMatrix_dev);
    cudaFree(degree_dev);

    return 0;
}


Overwriting listDegree.cu


In [6]:
%%shell
nvcc listDegree.cu -o listDegree
./listDegree

Adjacency Matrix:
0 1 1 0 0 
1 0 1 1 0 
1 1 0 1 1 
0 1 1 0 1 
0 0 1 1 0 
Node degrees:
Node 0: 2
Node 1: 3
Node 2: 4
Node 3: 3
Node 4: 2


In [7]:
%%writefile histogram.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <string>

#define N 5 // Number of nodes in the graph
#define MAX_DEGREE N // Maximum degree of a node in the graph

__global__ void computeDegree(int *adjMatrix, int *degree, int *histogram) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {
        int count = 0;
        for (int i = 0; i < N; ++i) {
            count += adjMatrix[id * N + i]; // Assuming the adjacency matrix is stored in row-major order
        }
        degree[id] = count;

        // Increment the corresponding histogram bin
        atomicAdd(&histogram[count], 1);
    }
}

int main() {
    // Read adjacency matrix from CSV file
    std::ifstream file("adjacency_matrix.csv");
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file!" << std::endl;
        return 1;
    }

    std::vector<std::vector<int>> adjMatrix(N, std::vector<int>(N, 0));
    std::string line;
    int row = 0;
    while (std::getline(file, line) && row < N) {
        std::stringstream ss(line);
        std::string cell;
        int col = 0;
        while (std::getline(ss, cell, ',') && col < N) {
            adjMatrix[row][col] = std::stoi(cell);
            col++;
        }
        row++;
    }

    // Allocate memory for adjacency matrix, degree array, and histogram
    int *adjMatrix_dev, *degree_dev, *histogram_dev;
    cudaMallocManaged(&adjMatrix_dev, N * N * sizeof(int));
    cudaMallocManaged(&degree_dev, N * sizeof(int));
    cudaMallocManaged(&histogram_dev, (MAX_DEGREE + 1) * sizeof(int)); // Maximum degree + 1 to include degrees from 0 to MAX_DEGREE

    // Copy adjacency matrix to device memory
    for (int i = 0; i < N; ++i) {
        cudaMemcpy(adjMatrix_dev + i * N, &adjMatrix[i][0], N * sizeof(int), cudaMemcpyHostToDevice);
    }

    // Initialize histogram array to zeros
    cudaMemset(histogram_dev, 0, (MAX_DEGREE + 1) * sizeof(int));

    // Launch kernel to compute degree and histogram
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    computeDegree<<<numBlocks, blockSize>>>(adjMatrix_dev, degree_dev, histogram_dev);
    cudaDeviceSynchronize();

    // Copy degrees and histogram from device to host
    int degree[N];
    int histogram[MAX_DEGREE + 1];
    cudaMemcpy(degree, degree_dev, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(histogram, histogram_dev, (MAX_DEGREE + 1) * sizeof(int), cudaMemcpyDeviceToHost);

    // Print out degrees and histogram
    std::cout << "Node degrees:" << std::endl;
    for (int i = 0; i < N; ++i) {
        std::cout << "Node " << i << ": " << degree[i] << std::endl;
    }

    std::cout << "\nHistogram of degrees:" << std::endl;
    for (int i = 0; i <= MAX_DEGREE; ++i) {
        std::cout << "Degree " << i << ": " << histogram[i] << std::endl;
    }

    // Free memory
    cudaFree(adjMatrix_dev);
    cudaFree(degree_dev);
    cudaFree(histogram_dev);

    return 0;
}

Writing histogram.cu


In [8]:
%%shell
nvcc histogram.cu -o histogram
./histogram

Node degrees:
Node 0: 2
Node 1: 3
Node 2: 4
Node 3: 3
Node 4: 2

Histogram of degrees:
Degree 0: 0
Degree 1: 0
Degree 2: 2
Degree 3: 2
Degree 4: 1
Degree 5: 0


In [9]:
%%writefile diameter.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <string>

#define N 5 // Number of nodes in the graph
#define INF 99999 // Infinity value for unreachable nodes

__global__ void floydWarshall(int *adjMatrix, int *dist) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int idy = threadIdx.y + blockIdx.y * blockDim.y;

    if (idx < N && idy < N) {
        if (idx == idy)
            dist[idx * N + idy] = 0;
        else if (adjMatrix[idx * N + idy] == 0)
            dist[idx * N + idy] = INF;
        else
            dist[idx * N + idy] = adjMatrix[idx * N + idy];
    }
    __syncthreads();

    for (int k = 0; k < N; k++) {
        if (dist[idx * N + k] + dist[k * N + idy] < dist[idx * N + idy]) {
            dist[idx * N + idy] = dist[idx * N + k] + dist[k * N + idy];
        }
        __syncthreads();
    }
}

int main() {
    // Read adjacency matrix from CSV file
    std::ifstream file("adjacency_matrix.csv");
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file!" << std::endl;
        return 1;
    }

    std::vector<std::vector<int>> adjMatrix(N, std::vector<int>(N, 0));
    std::string line;
    int row = 0;
    while (std::getline(file, line) && row < N) {
        std::stringstream ss(line);
        std::string cell;
        int col = 0;
        while (std::getline(ss, cell, ',') && col < N) {
            adjMatrix[row][col] = std::stoi(cell);
            col++;
        }
        row++;
    }

    // Allocate memory for adjacency matrix and distance matrix
    int *adjMatrix_dev, *dist_dev;
    cudaMallocManaged(&adjMatrix_dev, N * N * sizeof(int));
    cudaMallocManaged(&dist_dev, N * N * sizeof(int));

    // Copy adjacency matrix to device memory
    for (int i = 0; i < N; ++i) {
        cudaMemcpy(adjMatrix_dev + i * N, &adjMatrix[i][0], N * sizeof(int), cudaMemcpyHostToDevice);
    }

    // Launch kernel to run Floyd-Warshall algorithm
    dim3 threadsPerBlock(N, N);
    dim3 numBlocks(1, 1);
    if (N * N > 256) {
        threadsPerBlock.x = 256;
        threadsPerBlock.y = 1;
        numBlocks.x = (N + threadsPerBlock.x - 1) / threadsPerBlock.x;
        numBlocks.y = (N + threadsPerBlock.y - 1) / threadsPerBlock.y;
    }
    floydWarshall<<<numBlocks, threadsPerBlock>>>(adjMatrix_dev, dist_dev);
    cudaDeviceSynchronize();

    // Find the maximum distance (diameter) in the distance matrix
    int diameter = 0;
    for (int i = 0; i < N * N; i++) {
        if (dist_dev[i] != INF && dist_dev[i] > diameter)
            diameter = dist_dev[i];
    }

    std::cout << "Diameter of the graph: " << diameter << std::endl;

    // Free memory
    cudaFree(adjMatrix_dev);
    cudaFree(dist_dev);

    return 0;
}

Writing diameter.cu


In [10]:
%%shell
nvcc diameter.cu -o diameter
./diameter

Diameter of the graph: 2


In [11]:
%%writefile maxminDegree.cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <string>

#define N 5 // Number of nodes in the graph

__global__ void computeDegree(int *adjMatrix, int *degree) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < N) {
        int count = 0;
        for (int i = 0; i < N; ++i) {
            count += adjMatrix[id * N + i]; // Assuming the adjacency matrix is stored in row-major order
        }
        degree[id] = count;
    }
}

int main() {
    // Read adjacency matrix from CSV file
    std::ifstream file("adjacency_matrix.csv");
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file!" << std::endl;
        return 1;
    }

    std::vector<std::vector<int>> adjMatrix(N, std::vector<int>(N, 0));
    std::string line;
    int row = 0;
    while (std::getline(file, line) && row < N) {
        std::stringstream ss(line);
        std::string cell;
        int col = 0;
        while (std::getline(ss, cell, ',') && col < N) {
            adjMatrix[row][col] = std::stoi(cell);
            col++;
        }
        row++;
    }

    // Allocate memory for adjacency matrix and degree array
    int *adjMatrix_dev, *degree_dev;
    cudaMallocManaged(&adjMatrix_dev, N * N * sizeof(int));
    cudaMallocManaged(&degree_dev, N * sizeof(int));

    // Copy adjacency matrix to device memory
    for (int i = 0; i < N; ++i) {
        cudaMemcpy(adjMatrix_dev + i * N, &adjMatrix[i][0], N * sizeof(int), cudaMemcpyHostToDevice);
    }

    // Launch kernel to compute degree
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    computeDegree<<<numBlocks, blockSize>>>(adjMatrix_dev, degree_dev);
    cudaDeviceSynchronize();

    // Find minimum and maximum degrees along with their corresponding vertex IDs
    int minDegree = degree_dev[0], maxDegree = degree_dev[0];
    int minVertexID = 0, maxVertexID = 0;
    for (int i = 1; i < N; ++i) {
        if (degree_dev[i] < minDegree) {
            minDegree = degree_dev[i];
            minVertexID = i;
        }
        if (degree_dev[i] > maxDegree) {
            maxDegree = degree_dev[i];
            maxVertexID = i;
        }
    }

    // Print minimum and maximum degrees with their corresponding vertex IDs
    std::cout << "Minimum Degree: " << minDegree << " (Vertex ID: " << minVertexID << ")" << std::endl;
    std::cout << "Maximum Degree: " << maxDegree << " (Vertex ID: " << maxVertexID << ")" << std::endl;

    // Free memory
    cudaFree(adjMatrix_dev);
    cudaFree(degree_dev);

    return 0;
}

Writing maxminDegree.cu


In [12]:
%%shell
nvcc maxminDegree.cu -o maxminDegree
./maxminDegree

Minimum Degree: 2 (Vertex ID: 0)
Maximum Degree: 4 (Vertex ID: 2)
